# 預測是否會土石流

# import packages

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz
import pydotplus
import numpy as np


# Attributes and Features

*坡度         -> 0~30度
*土砂原料     -> 多:1，少:0
*累積雨量     -> 0~400毫米
*人為開發程度 -> 高:1，低:0
*地質年代     -> 年輕:1，老:0

# generate data

In [2]:
data=pd.DataFrame(np.random.randint(0,30,size=(1500,1)),columns = ['gradient'])
data['raw material']=np.random.choice(list({'0','1'}),len(data))
data['human']=np.random.choice(list({'0','1'}),len(data))
data['raw material']=np.random.choice(list({'0','1'}),len(data))
data['geology']=np.random.choice(list({'0','1'}),len(data))
data

,gradient,raw material,human,geology
0,27,1,1,0
1,19,0,1,0
2,2,0,0,1
3,5,1,0,0
4,23,0,1,1
5,6,1,0,0
6,1,1,0,1
7,27,1,0,0
8,20,1,1,1
9,20,1,0,1


In [ ]:
data['rain'] = np.random.normal(0, 400, len(data))
data

In [6]:
def pos(x):
    if x<0:
        return float(abs(x))
    return float(x)
data.rain=data.rain.apply(pos)

In [7]:
data.head()

,gradient,raw material,human,geology,rain
0,27,1,1,0,650.559199
1,19,0,1,0,116.002018
2,2,0,0,1,270.352014
3,5,1,0,0,528.061775
4,23,0,1,1,380.106347


evaluate the 'right' result

In [9]:
def mapping(x):
    if x.rain>=360:
        if x.gradient>=15 and x.gradient<=30:
            return 'may'
        else:
            if x['raw material']==1:
                return 'may'
            else:
                if x['geology']==1:
                    return 'may'
                else:
                    if x['human']==1:
                        return 'may'
                    else:
                        return 'maynot'
    else:
        if x.gradient<15:
            return 'maynot'
        else:
            if x['raw material']==1:
                return 'may'
            else: 
                return 'maynot'          

In [10]:
data['happen']=data.apply(mapping,axis=1)

In [11]:
data

,gradient,raw material,human,geology,rain,happen
0,27,1,1,0,650.559199,may
1,19,0,1,0,116.002018,maynot
2,2,0,0,1,270.352014,maynot
3,5,1,0,0,528.061775,maynot
4,23,0,1,1,380.106347,may
5,6,1,0,0,269.884942,maynot
6,1,1,0,1,598.719743,maynot
7,27,1,0,0,151.116415,maynot
8,20,1,1,1,278.180118,maynot
9,20,1,0,1,173.635888,maynot


In [12]:
def happen(x):
    if x=='may':
        x=1
    elif x=='maynot':
        x=0
    return x
data.happen=data.happen.apply(happen)

In [13]:
data.head()

,gradient,raw material,human,geology,rain,happen
0,27,1,1,0,650.559199,1
1,19,0,1,0,116.002018,0
2,2,0,0,1,270.352014,0
3,5,1,0,0,528.061775,0
4,23,0,1,1,380.106347,1


# Seperate data into training set and testing(validation) set

In [15]:
train = data[:800]
train.head()

,gradient,raw material,human,geology,rain,happen
0,27,1,1,0,650.559199,1
1,19,0,1,0,116.002018,0
2,2,0,0,1,270.352014,0
3,5,1,0,0,528.061775,0
4,23,0,1,1,380.106347,1


In [16]:
test = data[800:]
test.head()

,gradient,raw material,human,geology,rain,happen
800,0,0,1,1,120.686520,0
801,21,1,1,1,514.600300,1
802,13,0,1,1,19.219355,0
803,5,1,1,0,354.952919,0
804,20,1,0,1,606.082294,1


seperate attributes and answer

In [17]:
y = train['happen'].values
train = train.drop('happen', 1)

y

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,

# Build The Decision Tree

In [19]:
dtree=DecisionTreeClassifier(max_depth=5)
dtree.fit(train,y)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(train),
                class_names=['may_happen','may_not_happen'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("decisiontree.pdf")

True

In [20]:
dtree.feature_importances_

array([0.59703239, 0.        , 0.        , 0.        , 0.40296761])

# calculate accuracy

In [21]:
y_test = test['happen'].values
X_test = test.drop('happen', 1)

y_predict = dtree.predict(X_test)

y_predict

array([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,

In [22]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

0.9985714285714286